In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import FileSearchTool, VectorStoreDataSource, VectorStoreDataSourceAssetType
from azure.identity import DefaultAzureCredential

In [2]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str='Your Azure AI Foundation Connection String',
)

In [3]:
with project_client:

    # [START upload_file_and_create_agent_with_file_search]
    # We will upload the local file to Azure and will use it for vector store creation.
    _, asset_uri = project_client.upload_file("./data/intro_rag.md")

    # create a vector store with no file and wait for it to be processed
    ds = VectorStoreDataSource(asset_identifier=asset_uri, asset_type=VectorStoreDataSourceAssetType.URI_ASSET)
    vector_store = project_client.agents.create_vector_store_and_poll(data_sources=[ds], name="sample_vector_store")
    print(f"Created vector store, vector store ID: {vector_store.id}")

    # create a file search tool
    file_search_tool = FileSearchTool(vector_store_ids=[vector_store.id])

    # notices that FileSearchTool as tool and tool_resources must be added or the assistant unable to search the file
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-assistant",
        instructions="You are helpful assistant",
        tools=file_search_tool.definitions,
        tool_resources=file_search_tool.resources,
    )
    # [END upload_file_and_create_agent_with_file_search]
    print(f"Created agent, agent ID: {agent.id}")

    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    message = project_client.agents.create_message(
        thread_id=thread.id, role="user", content="What is GraphRAG?"
    )
    print(f"Created message, message ID: {message.id}")

    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Created run, run ID: {run.id}")

    project_client.agents.delete_vector_store(vector_store.id)
    print("Deleted vector store")

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Uploading intro_rag.md (< 1 MB): 100%|██████████| 2.40k/2.40k [00:00<00:00, 7.13kB/s]




Created vector store, vector store ID: vs_jAA5pjNQswxXc6KSfr0DmtXw
Created agent, agent ID: asst_5IJN1tHBdEJQ9a8R4eZAcLmJ
Created thread, thread ID: thread_Loly85hYhB63tghRIWY9x1au
Created message, message ID: msg_JSjg0swtGxUYhymZf7DF4NzB
Created run, run ID: run_vcDXkB6Rz91Yft9antnOxyov
Deleted vector store
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_DFbr4reO3mVAqtrCnz7q95nP', 'object': 'thread.message', 'created_at': 1735099498, 'assistant_id': 'asst_5IJN1tHBdEJQ9a8R4eZAcLmJ', 'thread_id': 'thread_Loly85hYhB63tghRIWY9x1au', 'run_id': 'run_vcDXkB6Rz91Yft9antnOxyov', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'GraphRAG is an AI-based tool for content interpretation and search. It utilizes large language models (LLMs) to parse data and create a knowledge graph, enabling it to answer questions based on a user-provided private dataset. Its primary intended uses include supporting critical information discovery and analysis in scenarios involvin